Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Tutorial: Load TAXI data and enrich it with Weather data in Pandas DataFrame

Install azureml-opendatasets package

In [4]:
!pip uninstall -y azureml-opendatasets
!pip install azureml-opendatasets

Begin by creating a dataframe to hold the taxi data. When working in a non-Spark environment, Open Datasets only allows downloading one month of data at a time with certain classes to avoid MemoryError with large datasets. To download 2 months of taxi data, iteratively fetch one month at a time, and before appending it to green_taxi_df randomly sample 2000 records from the specific month to avoid bloating the dataframe.

In [6]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from azureml.opendatasets import NycTlcGreen


green_taxi_df = pd.DataFrame([])
start = datetime.strptime("1/1/2016", "%m/%d/%Y")
end = datetime.strptime("1/31/2016", "%m/%d/%Y")

for sample_month in range(2):
    temp_df_green = NycTlcGreen(
        start + relativedelta(months=sample_month),
        end + relativedelta(months=sample_month)).to_pandas_dataframe()
    green_taxi_df = green_taxi_df.append(temp_df_green.sample(2000))

ActivityStarted, to_pandas_dataframe
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2016/puMonth=1/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2016/puMonth=1/part-00119-tid-6037743401120983271-619c4849-c957-4290-a1b8-66832cb385b6-12538.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=5969.23 [ms]
ActivityCompleted: Activity=to_pandas_dataframe, HowEnded=Success, Duration=5970.56 [ms]
ActivityStarted, to_pandas_dataframe
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2016/puMonth=2/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2016/puMonth=2/part-00060-tid-6037743401120983271-619c4849-c957-4290-a1b8-66832cb385b6-12479.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=6457.69 [ms]
ActivityCompleted: Activity=to_pandas_dataframe, HowEnded=Success, Duration=6459.1 [ms]

Save a copy of the raw_columns name list for clean up at the last step.

In [8]:
raw_columns = list(green_taxi_df.columns)

<font color='red'>Get mean values of pickupLatitude and pickupLongitude</font>

In [10]:
info = green_taxi_df.describe()
info['pickupLatitude']['mean'], info['pickupLongitude']['mean']

Out[ 4 ]: (40.654190485000612, -73.770879159927375)

Drop the rows that both lat/long are NaN, especially all columns in the first row are NaN.

In [12]:
green_taxi_df = green_taxi_df.dropna(how='all', subset=['lpepPickupDatetime', 'pickupLatitude', 'pickupLongitude'])

Make all pickupLatitude and pickupLongitude be the center location of the city.

In [14]:
def set_lat(x):
    return info['pickupLatitude']['mean']
def set_long(x):
    return info['pickupLongitude']['mean']
green_taxi_df['pickupLatitude'] = green_taxi_df[['pickupLatitude']].apply(set_lat, axis=1)
green_taxi_df['pickupLongitude'] = green_taxi_df[['pickupLongitude']].apply(set_long, axis=1)
green_taxi_df.head(5)

Out[ 6 ]: 
 vendorID lpepPickupDatetime ... totalAmount tripType
793496 2 2016-01-16 21:07:48 ... 19.30 1.0
373142 2 2016-01-08 10:11:38 ... 37.80 1.0
402180 2 2016-01-08 21:44:08 ... 13.50 1.0
1357877 1 2016-01-04 21:59:23 ... 17.80 1.0
284994 2 2016-01-25 10:51:16 ... 11.16 1.0

[5 rows x 23 columns]

The original index can fail the initialization of class LocationTimeCustomerData at below, so this is a workaround to add a monotonically increasing id column.

In [16]:
green_taxi_df['idx'] = list(range(len(green_taxi_df.index)))
green_taxi_df_idx = green_taxi_df.set_index('idx')
green_taxi_df_idx.head(5)

Out[ 7 ]: 
 vendorID lpepPickupDatetime ... totalAmount tripType
idx ... 
0 2 2016-01-16 21:07:48 ... 19.30 1.0
1 2 2016-01-08 10:11:38 ... 37.80 1.0
2 2 2016-01-08 21:44:08 ... 13.50 1.0
3 1 2016-01-04 21:59:23 ... 17.80 1.0
4 2 2016-01-25 10:51:16 ... 11.16 1.0

[5 rows x 23 columns]

Initialize LocationTimeCustomerData using pandas dataframe green_taxi.

In [18]:
# This is a package in preview.

from azureml.opendatasets.accessories.location_data import LatLongColumn
from azureml.opendatasets.accessories.location_time_customer_data \
    import LocationTimeCustomerData
from azureml.opendatasets import NoaaIsdWeather


green_taxi = LocationTimeCustomerData(
    green_taxi_df_idx,
    LatLongColumn('pickupLatitude', 'pickupLongitude'),
    'lpepPickupDatetime')

Initialize NoaaIsdWeather class, get enricher from it, and enrich the taxi data without aggregation

In [20]:
weather = NoaaIsdWeather(
    cols=["temperature", "precipTime", "precipDepth", "snowDepth"],
    start_date=datetime(2016, 1, 1, 0, 0),
    end_date=datetime(2016, 2, 28, 23, 59))
weather_enricher = weather.get_enricher()
new_green_taxi, processed_weather = weather_enricher.enrich_customer_data_no_agg(
    customer_data_object=green_taxi,
    location_match_granularity=1,
    time_round_granularity='day')

ActivityStarted, get_enricher
ActivityCompleted: Activity=get_enricher, HowEnded=Success, Duration=1.69 [ms]
ActivityStarted, enrich_customer_data_no_agg
ActivityStarted, enrich
Target paths: ['/year=2016/month=1/', '/year=2016/month=2/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading ISDWeather/year=2016/month=1/part-00005-tid-6700213360605767691-4491b75c-f137-489b-b5df-4204b9326fda-110.c000.snappy.parquet under container isdweatherdatacontainer
Reading ISDWeather/year=2016/month=2/part-00011-tid-6700213360605767691-4491b75c-f137-489b-b5df-4204b9326fda-116.c000.snappy.parquet under container isdweatherdatacontainer
Done.
ActivityStarted, _get_closest_location_kdTree
ActivityCompleted: Activity=_get_closest_location_kdTree, HowEnded=Success, Duration=307.59 [ms]
ActivityCompleted: Activity=enrich, HowEnded=Success, Duration=168338.18 [ms]
ActivityCompleted: Activity=enrich_customer_data_no_agg, HowEnded=Success, Duration=168624.07 [ms]

Preview the pandas dataframe new_green_taxi.data

In [22]:
new_green_taxi.data.head(3)

Out[ 10 ]: 
 vendorID ... customer_join_timeutf6j
0 2 ... 2016-01-16
1 2 ... 2016-01-08
2 2 ... 2016-01-08

[3 rows x 25 columns]

Define a dict `aggregations` to define how to aggregate each field at a hour level. For `snowDepth` and `temperature` we'll take the mean and for `precipTime` and `precipDepth` we'll take the hourly maximum. Use the groupby() function along with the aggregations to group data.

In [24]:
aggregations = {
    "snowDepth": "mean",
    "precipTime": "max",
    "temperature": "mean",
    "precipDepth": "max"}

The keys (`public_rankgroup`, `public_join_time`, `customer_rankgroup`, `customer_join_time`) used by groupby() and later merge() must be hacked here due to the current design.

In [26]:
public_rankgroup = processed_weather.id

public_join_time = [
    s for s in list(processed_weather.data.columns)
    if s.startswith('ds_join_time')][0]

customer_rankgroup = weather_enricher.location_selector.customer_rankgroup

customer_join_time = [
    s for s in list(new_green_taxi.data.columns)
    if type(s) is str and s.startswith('customer_join_time')][0]

weather_df_grouped = processed_weather.data.groupby(by=[public_rankgroup, public_join_time]).agg(aggregations)
weather_df_grouped.head(3)

Out[ 12 ]: 
 snowDepth ... temperature
public_rankgroupe8k4d ds_join_timetud1w ... 
1 2016-01-01 0.0 ... 5.590625
 2016-01-02 0.0 ... 3.087500
 2016-01-03 0.0 ... 4.668750

[3 rows x 4 columns]

Join the final dataframe, and preview the joined result.

In [28]:
joined_dataset = new_green_taxi.data.merge(
    weather_df_grouped,
    left_on=[customer_rankgroup, customer_join_time],
    right_on=[public_rankgroup, public_join_time],
    how='left')

final_df = joined_dataset[raw_columns + [
    "temperature", "precipTime", "precipDepth", "snowDepth"]]
final_df.head(5)

Out[ 13 ]: 
 vendorID lpepPickupDatetime ... precipDepth snowDepth
0 2 2016-01-16 21:07:48 ... 79.0 0.000000
1 2 2016-01-08 10:11:38 ... 0.0 0.000000
2 2 2016-01-08 21:44:08 ... 0.0 0.000000
3 1 2016-01-04 21:59:23 ... 0.0 0.000000
4 2 2016-01-25 10:51:16 ... 0.0 64.888889

[5 rows x 27 columns]

Check the join success rate.

In [30]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 27 columns):
vendorID 4000 non-null int32
lpepPickupDatetime 4000 non-null datetime64[ns]
lpepDropoffDatetime 4000 non-null datetime64[ns]
passengerCount 4000 non-null int32
tripDistance 4000 non-null float64
puLocationId 0 non-null object
doLocationId 0 non-null object
pickupLongitude 4000 non-null float64
pickupLatitude 4000 non-null float64
dropoffLongitude 4000 non-null float64
dropoffLatitude 4000 non-null float64
rateCodeID 4000 non-null int32
storeAndFwdFlag 4000 non-null object
paymentType 4000 non-null int32
fareAmount 4000 non-null float64
extra 4000 non-null float64
mtaTax 4000 non-null float64
improvementSurcharge 4000 non-null object
tipAmount 4000 non-null float64
tollsAmount 4000 non-null float64
ehailFee 0 non-null float64
totalAmount 4000 non-null float64
tripType 4000 non-null float64
temperature 4000 non-null float64
precipTime 4000 non-null float64
precipDepth 4000 non-null float64
snowDepth 4000 non-null float64
dtypes: datetime64[ns](2), float64(17), int32(4), object(4)
memory usage: 812.5+ KB